In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import lsst.sims.photUtils as photUtils
import lsst.sims.catUtils as catUtils
import lsst.sims.GalSimInterface as galsimInterface
from lsst.obs.lsstSim import LsstSimMapper

In [3]:
from lsst.sims.utils import ObservationMetaData
from lsst.sims.catUtils.utils import ObservationMetaDataGenerator

/Users/lynnej/lsstsw/stack/DarwinX86/sims_catalogs/2.3.5.sims.rc1/python/lsst/sims/catalogs/db/dbConnection.py:508: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/lynnej/lsstsw/stack/DarwinX86/sims_catalogs/2.3.5.sims.rc1/python/lsst/sims/catalogs/db/dbConnection.py:508: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


In [4]:
class NoisyCatalog(galsimInterface.GalSimSSM):
    noise_and_background = galsimInterface.ExampleCCDNoise(addNoise=True, addBackground=True)

In [5]:
opsdb = '/Users/lynnej/opsim/db/minion_1016_sqlite.db'
generator = ObservationMetaDataGenerator(database=opsdb, driver='sqlite')

In [6]:
night = 203
query = 'select obsHistId, min(expMJD) from summary where night=%d and filter="r"' %(night)
res = generator.opsimdb.execute_arbitrary(query)
obsHistId = res[0][0]
expMJD_min = res[0][1]
query = 'select fieldRA, fieldDec, expMJD, rotSkyPos, filter, FWHMeff, fiveSigmaDepth, obsHistId ' \
    'from summary where night=%d and filter="r"' % night
res = generator.opsimdb.execute_arbitrary(query)
fieldRA = np.degrees(float(res[0][0]))
fieldDec = np.degrees(float(res[0][1]))
expMJD = float(res[0][2])
rotSkyPos = np.degrees(float(res[0][3]))
bandpass = str(res[0][4])
seeing = float(res[0][5])
m5 = float(res[0][6])
m5 = 24.5
obsHistId = int(res[0][7])
print(seeing, m5, expMJD, fieldRA, fieldDec, bandpass, obsHistId)
obs = ObservationMetaData(pointingRA = fieldRA, pointingDec = fieldDec,
                         boundType='circle', boundLength=2.0, 
                         mjd = expMJD, rotSkyPos = rotSkyPos,
                         bandpassName = bandpass, m5 = m5, seeing = seeing)

1.081626 24.5 59782.98551 248.506176989 -35.9013062598 r 151038


In [7]:
ssmObj = catUtils.baseCatalogModels.SolarSystemObj()
#cat = NoisyCatalog(ssmObj, obs)
cat = galsimInterface.GalSimSSM(ssmObj, obs_metadata=obs)
#starObj = catUtils.baseCatalogModels.StarObj()
#cat = galsimInterface.GalSimStars(starObj, obs_metadata=obs)
cat.camera = LsstSimMapper().camera
cat.allowed_chips = ['R:1,1 S:1,1', 'R:1,1 S:0,0']

In [8]:
cat.setPSF(galsimInterface.Kolmogorov_and_Gaussian_PSF())
#cat.setPSF(galsimInterface.DoubleGaussianPSF())
cat.write_catalog('ssm_test_catalog.txt')
cat.write_images(nameRoot='ssmImage')

['ssmImage_R_1_1_S_1_1_r.fits', 'ssmImage_R_1_1_S_0_0_r.fits']

In [9]:
import lsst.afw.image as afwImage
im = afwImage.ImageF('ssmImage_R_1_1_S_0_0_r.fits').getArray()
total_counts = im.sum()
print(total_counts)

40808.7


In [10]:
from lsst.sims.catalogs.definitions import InstanceCatalog
from lsst.obs.lsstSim import LsstSimMapper

class ic(InstanceCatalog, catUtils.mixins.AstrometrySSM, catUtils.mixins.CameraCoords):
    camera = LsstSimMapper().camera
    column_outputs = ['objid', 'raJ2000', 'decJ2000', 'chipName']
    transformations = {'raJ2000': np.degrees, 'decJ2000': np.degrees}
    default_formats = {'f':'%.13f'}

In [11]:
cc = ic(ssmObj, obs)
cc.write_catalog('testcat.txt')